In [1]:
import cv2
import numpy as np
import os

def extract_frames(video_path, output_dir):
    """Extract frames from a video and save them to a directory."""
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.png")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_dir}")
    return output_dir, frame_count

def compute_sharpness(frame):
    """Compute sharpness of a frame using the variance of the Laplacian."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray_frame, cv2.CV_64F).var()

def select_clear_frames(frame_dir, num_frames):
    """Select the sharpest frames from a directory."""
    frame_files = sorted([os.path.join(frame_dir, f) for f in os.listdir(frame_dir) if f.endswith(".png")])
    sharpness_scores = []

    for file in frame_files:
        frame = cv2.imread(file)
        sharpness = compute_sharpness(frame)
        sharpness_scores.append((file, sharpness))

    # Sort by sharpness in descending order and select top frames
    sharpness_scores.sort(key=lambda x: x[1], reverse=True)
    selected_frames = [cv2.imread(f[0]) for f in sharpness_scores[:num_frames]]

    print(f"Selected {num_frames} clearest frames")
    return selected_frames

def combine_frames_to_video(frames, output_video_path, fps):
    """Combine frames into a video."""
    if not frames:
        print("No frames provided for video creation.")
        return

    height, width, layers = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame in frames:
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved to {output_video_path}")

def process_video(input_video, output_video, temp_dir, num_frames):
    """Main function to process the video."""
    # Step 1: Extract frames from the video
    frame_dir, total_frames = extract_frames(input_video, temp_dir)

    # Step 2: Select the clearest frames
    selected_frames = select_clear_frames(frame_dir, min(num_frames, total_frames))

    # Step 3: Combine selected frames into a new video
    fps = 30  # Adjust to your desired frame rate
    combine_frames_to_video(selected_frames, output_video, fps)

# Usage
input_video_path = "sample.mp4"
output_video_path = "output_video.mp4"
temp_frames_dir = "temp_frames"
clear_frame_count = 200  # Number of clearest frames to select

process_video(input_video_path, output_video_path, temp_frames_dir, clear_frame_count)


KeyboardInterrupt: 